# Introduction and short technical overview of the Fluke 8846A

The aim of this notebook is to introduce the driver class *Fluke_8846A* utilizing the programmable digital multimeter (DMM) **Fluke 8846A** in my lab.

The **Fluke 8846A** comes with graphical color display for displaying measurement curves, histograms, bar graphs, statistics and mathematical functions. The **Fluke 8846A** 6.5 digit precision multimeter provides the accuracy and flexibility required for demanding measurements for laboratory and system applications.

More information on technical details, specifications and downloadable documentation can be found here:

- [8845A/8846A 6.5 Digit Precision Multimeters](https://us.flukecal.com/products/data-acquisition-and-test-equipment/bench-multimeters/8845a8846a-65-digit-precision-multime)

![Front view of the Fluke 8846A](images/fluke_8846a_front.jpg)  
Front view of the Fluke 8846A

The following **programming manual** was helpful in understanding how to communicate with the DMM via its SCPI commands:

- [8845A/8846A Programmers Manual (Eng) (923.05 KB)](https://download.flukecal.com/pub/literature/8845A___pmeng0300.pdf)

# Possibilities of communication with the Fluke 8846A

## Telnet-based communication (without Python)

As described in the programming manual, it is possible to communicate with the **Fluke 8846A** directly via the Telnet protocol. The connection is established by specifying the SCPI port:

```bash
$ telnet 192.168.12.134 3490

Trying 192.168.12.134...
Connected to 192.168.12.134.
Escape character is '^]'.
*IDN?
FLUKE,8846A,2034021,08/02/10-11:53
*RST
SYST:REM
CONF:TEMP:RTD
READ?
+1.72179000E+01
^]
telnet> Connection closed.
```

The SCPI or Telnet connection is terminated via `STRG+]` and then `STRG+D`.

## Socket-based communication (in Python)

For the class implementation for the **Rigol DP832A** and the DMM **Keysight 34465A** I already had very good experiences with the Python library *PyVisa*. Also because of the support of the standardized SCPI commands by the DMM **Fluke 8846A** *PyVisa* would have been my 1st choice.

However, unlike the Rigol and Keysight device, the Fluke DMM does not support the `INSTR` TCP stream - probably the Fluke firmware is just too old (it is from 2010 according to system information!). A newer firmware as well as instructions for firmware upgrade could not be found on the official Fluke pages so far. That's what I call a really weak product support!

Alternatively, *PyVisa* provides a socket stream for TCP connections (see [VISA Resource Syntax and Examples](https://pyvisa.readthedocs.io/en/1.8/names.html#visa-resource-syntax-and-examples)), e.g.:

```python
rm = ResourceManager('@py')
dmm_socket = rm.open_resource('TCPIP0::192.168.12.134::3490::SOCKET')
```

Writing SCPI commands worked with it, if they were terminated appropriately with `\n`. However, reading the return values ended with the error: `VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.`.  
Therefore I decided not to use *PyVisa* and to implement the communication via raw sockets with the Python library *socket* (as of 2022-06-02). The following section demonstrates the socket-based communication.

Among other sources, the use of raw sockets was inspired by: [ModuleForKeithley](https://gist.github.com/rinitha/0844a61a82006fe92c78)

### Basic example

In [1]:
import socket

In [2]:
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

try:
    dmm_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    dmm_sock.connect((dmm_ip, dmm_port))
    
except Exception as e: 
    print("Something's wrong with %s:%d. Exception is %s" % (dmm_ip, dmm_port, e))

In [3]:
# set timeout on blocking socket operations in [s]
dmm_sock.settimeout(0.5)

In [4]:
# get device information
scpi_msg = "*IDN?\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [5]:
reading = dmm_sock.recv(128) # buffer size is 128 bytes
print(reading.decode().strip())

FLUKE,8846A,2034021,08/02/10-11:53


In [19]:
# close the connection to the device
dmm_sock.close()

In [6]:
# reset the device
scpi_msg = "*RST\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [7]:
# get device into remote mode
scpi_msg = "SYST:REM\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [8]:
# configure device to resistance measurement
scpi_msg = "CONF:RES DEF\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [9]:
# read measurement value
scpi_msg = "READ?\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [10]:
reading = dmm_sock.recv(64) # buffer size is 64 bytes
print(reading.decode().strip())

+1.09975210E+02


In [11]:
# decode byte stream, strip whitespaces and newline characters from string and cast to float
res_flt = float(reading.decode().strip())

print("Resistor: {:.2f} Ohm".format(res_flt))

Resistor: 109.98 Ohm


### Example with use of the secondary display

In [12]:
# configure device to temperature measurement
# configure secondary display to read corresponding resistance value
scpi_msg = 'FUNC1 "TEMP:RTD"; FUNC2 "RES"\n'
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [13]:
# read out primary and secondary displays simultaneously
scpi_msg = "READ?;FETCH2?\n"
dmm_sock.sendall(scpi_msg.encode('utf-8'))

In [14]:
reading = dmm_sock.recv(64) # buffer size is 64 bytes

In [15]:
# decode byte stream, strip whitespaces and newline characters from string and split into list of values
ret_val_list = reading.decode().strip().split(';')

In [16]:
# iterate through the list and cast elements to float
for idx, val in enumerate(ret_val_list):
    ret_val_list[idx] = float(ret_val_list[idx])

In [18]:
print("Temperature: {:.7f} °C, resistance {:.7f} Ohm".format(ret_val_list[0], ret_val_list[1]))

Temperature: 25.5638000 °C, resistance 109.9533000 Ohm


### Socket-based wrapper class `Fluke_8846A`

The new wrapper class **Fluke_8846A** in the python file *Fluke_8846A_class.py* implements the communication with the DMM Fluke 8846A via LAN interface and SCPI commands using TCP sockets.

In [20]:
# import wrapper class Fluke_8846A from python file Fluke_8846A_class.py
from Fluke_8846A_class import Fluke_8846A

import time

In [21]:
# IP of devices
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

# create new device object for the digital multimeter (DMM) Fluke 8846A
dmm = Fluke_8846A(tcp_ip = dmm_ip, tcp_port = dmm_port)

In [22]:
# read connection state of the device
dmm.status

'Connected'

In [23]:
# read connection path (at the moment there is only TCP/IP implemented)
dmm.connected_with

'FLUKE 8846A over LAN on 192.168.12.134, port 3490'

In [24]:
# get device information
dmm.getDevInfos()

['FLUKE', '8846A', '2034021', '08/02/10-11:53']

In [38]:
# close the connection to the device
dmm.closeConnection()

In [26]:
# open the connection again
dmm.openConnection(tcp_ip = dmm_ip, tcp_port = dmm_port)

In [27]:
# get a list of valid configurations for temperature measurement
list(dmm.conf_measurement_dict.keys())

['00_RES',
 '01_FRES',
 '02_RTD',
 '03_FRTD',
 '04_RTD_RES',
 '05_FRTD_RES',
 '06_VOLT_AC',
 '07_VOLT_AC_FREQ',
 '08_VOLT_DC',
 '09_CURR_AC',
 '10_CURR_AC_FREQ',
 '11_CURR_DC',
 '12_CONT',
 '13_CAP']

In [28]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('01_FRES')

In [34]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('04_RTD_RES')

In [30]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('08_VOLT_DC')

In [31]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('09_CURR_AC')

In [32]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('12_CONT')

In [33]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('13_CAP')

In [35]:
# get current configuration
dmm.getConfig()

'"TEMP:RTD +0.000000E+00,+1.000000E-03"'

In [36]:
# get measurement with current configuration
dmm.getMeasurement()

{'temperature_value': 25.5309,
 'temperature_unit': '°C',
 'resistance_value': 109.9406,
 'resistance_unit': 'Ohm'}

### Sample application for temperature measurement

In this sample program, a temperature sensor Pt100 is read out continuously using the wrapper class `Fluke_8846A`.

The class `MeasExecTimeOfProgram` is additionally used for performance measurement.

In [2]:
import time

# import wrapper class Fluke_8846A from python file Fluke_8846A_class.py
from Fluke_8846A_class import Fluke_8846A

# import class MeasExecTimeOfProgram from python file MeasExecTimeOfProgramclass.py
from MeasExecTimeOfProgram_class import MeasExecTimeOfProgram

INTERVAL = 0.2

# IP of devices
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

# create new device object for the digital multimeter (DMM) Fluke 8846A
dmm = Fluke_8846A(tcp_ip = dmm_ip, tcp_port = dmm_port)

# initiate measuring execution time
execTime = MeasExecTimeOfProgram()
execTime.initLogger()

# get device information
dmm.getDevInfos()

# configure DMM for temperature measurement with sensor type Pt100
# with its corresponding resistance value on the secondary display
dmm.confMeasurement('04_RTD_RES')

while True:
    try:
        execTime.start()
        
        # get measurement with current configuration
        ret_dict = dmm.getMeasurement()
        
        deltaTime = execTime.stop()
        execTime.addSample(deltaTime)
        
        print("<{:s}> Temperature: {:.3f} {}, Resistance: {:.3f} {}, Execution time: {:.6f}".format(time.strftime('%H:%M:%S'), 
                                                                                                    ret_dict['temperature_value'], 
                                                                                                    ret_dict['temperature_unit'],
                                                                                                    ret_dict['resistance_value'],
                                                                                                    ret_dict['resistance_unit'],
                                                                                                    deltaTime))
            
        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        # close the connection to the device
        dmm.closeConnection()
        break

<22:17:00> Temperature: 30.114 °C, Resistance: 111.717 Ohm, Execution time: 936.740160
<22:17:01> Temperature: 30.117 °C, Resistance: 111.718 Ohm, Execution time: 626.561403
<22:17:02> Temperature: 30.116 °C, Resistance: 111.718 Ohm, Execution time: 627.581120
<22:17:03> Temperature: 30.117 °C, Resistance: 111.718 Ohm, Execution time: 627.056122
<22:17:03> Temperature: 30.118 °C, Resistance: 111.719 Ohm, Execution time: 626.888037
<22:17:04> Temperature: 30.119 °C, Resistance: 111.719 Ohm, Execution time: 626.461029
<22:17:05> Temperature: 30.117 °C, Resistance: 111.718 Ohm, Execution time: 626.589298
<22:17:06> Temperature: 30.119 °C, Resistance: 111.719 Ohm, Execution time: 626.571178
<22:17:07> Temperature: 30.117 °C, Resistance: 111.718 Ohm, Execution time: 628.631592
<22:17:08> Temperature: 30.117 °C, Resistance: 111.718 Ohm, Execution time: 626.412153
<22:17:08> Temperature: 30.117 °C, Resistance: 111.718 Ohm, Execution time: 626.295328
<22:17:09> Temperature: 30.113 °C, Resistan

In [3]:
execTime.getLogBuffer()

,Time samples [ms]
0,936.740160
1,626.561403
2,627.581120
3,627.056122
4,626.888037
5,626.461029
6,626.589298
7,626.571178
8,628.631592
9,626.412153


In [14]:
execTime.getStatistics()

count     12.000000
mean     626.818041
std        0.681428
min      626.219273
25%      626.448810
50%      626.566291
75%      626.930058
max      628.631592
Name: Time samples [ms], dtype: float64

## PyVisa based communication (update)

**Update (2022-06-12):**  
Found the solution now: *PyVisa* must be made aware of the device-specific end-of-line terminations. The **Fluke 8846A** expects `\n` termination of SCPI commands for both directions (read and write) according to the programming manual.
The following documentation was helpful here:

- [PyVisa: Termination characters](https://pyvisa.readthedocs.io/en/latest/introduction/resources.html#termination-characters)
- [PyVisa: Getting the instrument configuration right](https://pyvisa.readthedocs.io/en/latest/introduction/communication.html#getting-the-instrument-configuration-right)
- [Python: Connect device using Visa TCP Socket connection](https://stackoverflow.com/questions/65630897/python-connect-device-using-visa-tcp-socket-connection)

### PyVisa-based basic example

In [47]:
import pyvisa
 
try:  
  rm = pyvisa.ResourceManager('@py') 
  pyvisa.log_to_screen
  dmm_socket = 'TCPIP0::192.168.12.134::3490::SOCKET'
  dmm = rm.open_resource(dmm_socket)
  print('Open Successful!')
 
except Exception as e:

    print('[!] Exception:' +str(e))

Open Successful!


In [53]:
# close connection
dmm.close()

In [48]:
# inspired by https://stackoverflow.com/questions/65630897/python-connect-device-using-visa-tcp-socket-connection
dmm.read_termination = '\n'
dmm.write_termination = '\n'

print('IDN:' +str(dmm.query('*IDN?')))

IDN:FLUKE,8846A,2034021,08/02/10-11:53


In [49]:
# reset device
cmd = '*RST'
dmm.write(cmd)

5

In [50]:
# get device into remote mode
cmd = "SYST:REM"
dmm.write(cmd)

9

In [51]:
# select temperature measurement
cmd = "CONF:TEMP:RTD"
dmm.write(cmd)

14

In [52]:
# get measurement
cmd = 'READ?'
ret_val = dmm.query(cmd)

# strip whitespaces and newline characters from string and cast to float
ret_val = ret_val.strip()
ret_val = float(ret_val)

print("Temperature: {:.6f} °C".format(ret_val))

Temperature: 27.729100 °C


### PyVisa-based wrapper class `PyVisa_Fluke_8846A`

In [2]:
# import wrapper class Fluke_8846A from python file Fluke_8846A_class.py
from PyVisa_Fluke_8846A_class import PyVisa_Fluke_8846A

import time

In [3]:
# IP of devices
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

# create new device object for the digital multimeter (DMM) Fluke 8846A
dmm = PyVisa_Fluke_8846A(tcp_ip = dmm_ip, tcp_port = dmm_port)

In [9]:
# read connection state of the device
dmm.status

'Connected'

In [10]:
# read connection path (at the moment there is only TCP/IP implemented)
dmm.connected_with

'FLUKE 8846A over LAN on 192.168.12.134, port 3490'

In [11]:
# get device information
dmm.getDevInfos()

['FLUKE', '8846A', '2034021', '08/02/10-11:53']

In [7]:
# close the connection to the device
dmm.closeConnection()

In [8]:
# open the connection again
dmm.openConnection(tcp_ip = dmm_ip, tcp_port = dmm_port)

In [12]:
# get a list of valid configurations for temperature measurement
list(dmm.conf_measurement_dict.keys())

['00_RES',
 '01_FRES',
 '02_RTD',
 '03_FRTD',
 '04_RTD_RES',
 '05_FRTD_RES',
 '06_VOLT_AC',
 '07_VOLT_AC_FREQ',
 '08_VOLT_DC',
 '09_CURR_AC',
 '10_CURR_AC_FREQ',
 '11_CURR_DC',
 '12_CONT',
 '13_CAP']

In [28]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('01_FRES')

In [13]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('04_RTD_RES')

In [30]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('08_VOLT_DC')

In [31]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('09_CURR_AC')

In [32]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('12_CONT')

In [33]:
# configure DMM for measurement with a valid configuration
dmm.confMeasurement('13_CAP')

In [14]:
# get current configuration
dmm.getConfig()

'"TEMP:RTD +0.000000E+00,+1.000000E-03"'

In [16]:
# get measurement with current configuration
dmm.getMeasurement()

{'temperature_value': 25.6906,
 'temperature_unit': '°C',
 'resistance_value': 110.0025,
 'resistance_unit': 'Ohm'}

### Sample application for temperature measurement

In this sample program, a temperature sensor Pt100 is read out continuously using the wrapper class `PyVisa_Fluke_8846A`.

The class `MeasExecTimeOfProgram` is additionally used for performance measurement.

In [15]:
import time

# import wrapper class PyVisa_Fluke_8846A from python file PyVisa_Fluke_8846A_class.py
from PyVisa_Fluke_8846A_class import PyVisa_Fluke_8846A

# import class MeasExecTimeOfProgram from python file MeasExecTimeOfProgramclass.py
from MeasExecTimeOfProgram_class import MeasExecTimeOfProgram

INTERVAL = 0.2

# IP of devices
#dmm_ip = "192.168.10.117"
dmm_ip = "192.168.12.134"

# port for SCPI connection
dmm_port = 3490

# create new device object for the digital multimeter (DMM) Fluke 8846A
dmm = PyVisa_Fluke_8846A(tcp_ip = dmm_ip, tcp_port = dmm_port)

# initiate measuring execution time
execTime = MeasExecTimeOfProgram()
execTime.initLogger()

# get device information
dmm.getDevInfos()

# configure DMM for temperature measurement with sensor type Pt100
# with its corresponding resistance value on the secondary display
dmm.confMeasurement('04_RTD_RES')

while True:
    try:
        execTime.start()
        
        # get measurement with current configuration
        ret_dict = dmm.getMeasurement()
        
        deltaTime = execTime.stop()
        execTime.addSample(deltaTime)
        
        print("<{:s}> Temperature: {:.3f} {}, Resistance: {:.3f} {}, Execution time: {:.6f}".format(time.strftime('%H:%M:%S'), 
                                                                                                    ret_dict['temperature_value'], 
                                                                                                    ret_dict['temperature_unit'],
                                                                                                    ret_dict['resistance_value'],
                                                                                                    ret_dict['resistance_unit'],
                                                                                                    deltaTime))
            
        time.sleep(INTERVAL)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        # close the connection to the device
        dmm.closeConnection()
        break

<22:17:55> Temperature: 30.161 °C, Resistance: 111.735 Ohm, Execution time: 644.728661
<22:17:56> Temperature: 30.159 °C, Resistance: 111.734 Ohm, Execution time: 607.444286
<22:17:57> Temperature: 30.159 °C, Resistance: 111.734 Ohm, Execution time: 613.839865
<22:17:58> Temperature: 30.160 °C, Resistance: 111.735 Ohm, Execution time: 607.913494
<22:17:59> Temperature: 30.159 °C, Resistance: 111.734 Ohm, Execution time: 600.943089
<22:17:59> Temperature: 30.158 °C, Resistance: 111.734 Ohm, Execution time: 627.319098
<22:18:00> Temperature: 30.160 °C, Resistance: 111.735 Ohm, Execution time: 609.814405
<22:18:01> Temperature: 30.163 °C, Resistance: 111.736 Ohm, Execution time: 614.394426
<22:18:02> Temperature: 30.158 °C, Resistance: 111.734 Ohm, Execution time: 608.453274
<22:18:03> Temperature: 30.160 °C, Resistance: 111.735 Ohm, Execution time: 607.698679
<22:18:03> Temperature: 30.163 °C, Resistance: 111.736 Ohm, Execution time: 599.097013
<22:18:04> Temperature: 30.163 °C, Resistan

In [16]:
execTime.getLogBuffer()

,Time samples [ms]
0,644.728661
1,607.444286
2,613.839865
3,607.913494
4,600.943089
5,627.319098
6,609.814405
7,614.394426
8,608.453274
9,607.698679


In [22]:
execTime.getStatistics()

count     15.000000
mean     610.504389
std        6.789675
min      599.097013
25%      607.806087
50%      608.453274
75%      613.555312
max      627.319098
Name: Time samples [ms], dtype: float64